<a href="https://colab.research.google.com/github/ganeshred/NLPAuthorshipAttributionNLM/blob/main/POS_tags%2B_glove_Task2_bi_LSTM_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torchtext==0.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.8 MB/s eta 0:00:00
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.15.1
    Uninstalling torchtext-0.15.1:
      Successfully uninstalled torchtext-0.15.1


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('/content/gdrive/MyDrive/balanced_task2.csv')

In [5]:
df.head()

,Unnamed: 0,text,class
0,664,Netanyahu barely scratches out a victory in Is...,0
1,61,"In ""Hunters,"" the premiere episode of a new se...",0
2,518,. cricket ice cream is a luxury ice cream made...,0
3,248,Graciela Iturbide was born in Mexico City in 1...,0
4,435,A new study from the University of Rochester d...,0


In [7]:
import nltk
from nltk.tokenize import word_tokenize

# Example texts
texts = df['text']

# Example labels
labels = df['class']



In [8]:
# load GloVe vectors
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader


# Un comment this code to download glove 50d vector txt file
# import os
# import urllib.request

# # Set the URL and file paths
# url = 'http://nlp.stanford.edu/data/glove.6B.zip'
# file_path = './glove.6B.zip'
# extract_path = '/content/gdrive/MyDrive/NLP/glove.6B'

# # Download the file
# if not os.path.exists(file_path):
#     urllib.request.urlretrieve(url, file_path)

# # Extract the file
# if not os.path.exists(extract_path):
#     os.makedirs(extract_path)
#     os.system(f'unzip {file_path} -d {extract_path}')



In [9]:
# Load the embeddings into a dictionary
import os
embeddings_dict = {}
with open(os.path.join('/content/gdrive/MyDrive/NLP/glove.6B/glove.6B.50d.txt'), 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        embeddings_dict[word] = vector

In [10]:
# create vocabulary and index-to-token mappings for GloVe words
glove_vocab = {'<PAD>': 0, '<UNK>': 1}
glove_vectors_list = [np.zeros(50), np.random.normal(scale=0.6, size=50)]
# print(glove_vectors_list)
for word in embeddings_dict:
    glove_vocab[word] = len(glove_vocab)
    glove_vectors_list.append(embeddings_dict[word])
glove_vectors_tensor = torch.FloatTensor(glove_vectors_list)


<ipython-input-10-9496caace8fa>:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  glove_vectors_tensor = torch.FloatTensor(glove_vectors_list)


In [11]:
glove_vocab['the']

2

In [12]:
glove_vectors_tensor.size()

torch.Size([400002, 50])

In [13]:
glove_vectors_tensor[3]

tensor([ 0.0134,  0.2368, -0.1690,  0.4095,  0.6381,  0.4771, -0.4285, -0.5564,
        -0.3640, -0.2394,  0.1300, -0.0637, -0.3957, -0.4816,  0.2329,  0.0902,
        -0.1332,  0.0786, -0.4163, -0.1543,  0.1007,  0.4889,  0.3123, -0.1252,
        -0.0375, -1.5179,  0.1261, -0.0244, -0.0430, -0.2835,  3.5416, -0.1196,
        -0.0145, -0.1499,  0.2186, -0.3341, -0.1387,  0.3181,  0.7036,  0.4486,
        -0.0803,  0.6300,  0.3211, -0.4676,  0.2279,  0.3603, -0.3782, -0.5666,
         0.0447,  0.3039])

In [14]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [15]:
import nltk
from nltk.tokenize import word_tokenize

pos_tagged_texts = []
for i in range(len(texts)):
    text = texts[i]
    label = labels[i]
    if text is not None and not isinstance(text, float): # make sure text is not None or NaN
      tokens = word_tokenize(text)
      pos_tags = nltk.pos_tag(tokens)
      pos_tagged_text = [(word, pos) for (word, pos) in pos_tags]
      pos_tagged_texts.append((pos_tagged_text, tokens, label))


In [16]:
for i in pos_tagged_texts[0]:
  print(i)

[('Netanyahu', 'NNP'), ('barely', 'RB'), ('scratches', 'VBZ'), ('out', 'RP'), ('a', 'DT'), ('victory', 'NN'), ('in', 'IN'), ('Israeli', 'NNP'), ('elections', 'NNS')]
['Netanyahu', 'barely', 'scratches', 'out', 'a', 'victory', 'in', 'Israeli', 'elections']
0


In [17]:
pos_to_index = {pos: i for i, pos in enumerate(set([pos for text, _, label in pos_tagged_texts for word, pos in text]))}
label_to_index = {label: i for i, label in enumerate(set([label for text, _, label in pos_tagged_texts]))}

In [18]:
pos_to_index

{'CC': 0,
 ':': 1,
 '$': 2,
 'WRB': 3,
 'VBG': 4,
 'DT': 5,
 'PDT': 6,
 'FW': 7,
 'VBZ': 8,
 '#': 9,
 'IN': 10,
 'EX': 11,
 'NNPS': 12,
 '(': 13,
 'VBN': 14,
 '.': 15,
 'VBP': 16,
 'RBR': 17,
 'RP': 18,
 'RBS': 19,
 'JJ': 20,
 'POS': 21,
 'RB': 22,
 'WP': 23,
 'PRP': 24,
 'PRP$': 25,
 'JJR': 26,
 ')': 27,
 ',': 28,
 'WP$': 29,
 'NN': 30,
 'NNS': 31,
 'WDT': 32,
 '``': 33,
 'VBD': 34,
 'MD': 35,
 'NNP': 36,
 'TO': 37,
 'CD': 38,
 'UH': 39,
 "''": 40,
 'JJS': 41,
 'SYM': 42,
 'VB': 43}

In [19]:
label_to_index

{0: 0, 1: 1}

In [20]:
from torch.nn.utils.rnn import pad_sequence

import torch
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    # print(len(batch[0]))
    # for i in batch[0]:
    # print(batch[0][1])
      
    pos_indices = [torch.tensor(sample[0], dtype=torch.long) for sample in batch]
    pos_indices_padded = pad_sequence(pos_indices, batch_first=True, padding_value=0)

    text_indices = [torch.tensor(sample[1], dtype=torch.long) for sample in batch]
    text_indices_padded = pad_sequence(pos_indices, batch_first=True, padding_value=0)
    
    # for sample in batch:
    #   for i in sample:
    #     print(sample)

    labels = torch.tensor([sample[2] for sample in batch], dtype=torch.long)
    return pos_indices_padded,text_indices_padded, labels


class POSTaggedTextDataset(Dataset):
    def __init__(self, pos_tagged_texts, pos_to_index, glove_embeddings,glove_list, label_to_index):
        self.pos_tagged_texts = pos_tagged_texts
        self.pos_to_index = pos_to_index
        self.glove_embeddings = glove_embeddings
        self.glove_list = glove_list
        self.label_to_index = label_to_index
        
    def __len__(self):
        return len(self.pos_tagged_texts)
    
    def __getitem__(self, index):
        postext,text, label = self.pos_tagged_texts[index]
        
        # Convert POS tags to indices
        pos_indices = [self.pos_to_index[pos] for _, pos in postext]
        
        # Convert text to GloVe embeddings
        # print(self.glove_embeddings[self.glove_list['word']])
        glove_input = [self.glove_list.get(word,self.glove_list['<UNK>']) for word in text]
        
        # Convert label to index
        label_index = self.label_to_index[label]
        
        return pos_indices, glove_input, label_index





In [21]:
pos_to_index = {pos: i for i, pos in enumerate(set([pos for text,_, label in pos_tagged_texts for word, pos in text]))}
label_to_index = {label: i for i, label in enumerate(set([label for text,_, label in pos_tagged_texts]))}

In [22]:
dataset = POSTaggedTextDataset(pos_tagged_texts, pos_to_index,glove_vectors_list,glove_vocab, label_to_index)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

In [23]:
# for i in dataloader:
#   # print(len(i))
#   for j in i:
#     print(j.size())
#   break

In [24]:
import torch.nn.functional as F



class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()

    def forward(self, prediction_text, prediction_pos, target_text):
        # predicted_pos: (batch_size, seq_len, num_classes)
        # target_pos: (batch_size, seq_len)
        loss_text = F.cross_entropy(prediction_text.view(-1, prediction_text.size(-1)), target_text.view(-1))
        
        # Compute cross-entropy loss for POS prediction
        loss_pos = F.cross_entropy(prediction_pos.view(-1, prediction_pos.size(-1)), target_text.view(-1))
        
        # Return total loss
        total_loss = loss_text + loss_pos
        
        return total_loss


# def custom_loss(prediction_text, prediction_pos, target_text):
#     # Compute cross-entropy loss for text prediction
#     loss_text = F.cross_entropy(prediction_text.view(-1, prediction_text.size(-1)), target_text.view(-1))
#     # Compute cross-entropy loss for POS prediction
#     loss_pos = F.cross_entropy(prediction_pos.view(-1, prediction_pos.size(-1)), target_text.view(-1))
#     # Return total loss
#     total_loss = loss_text + loss_pos
#     return total_loss


In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader


class POS_GloVe_LSTM(nn.Module):
    def __init__(self, text_vocab_size, pos_vocab_size, embedding_dim, hidden_dim, num_layers, dropout, output_size):
        super(POS_GloVe_LSTM, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        # Embedding layer for text field using GloVe embeddings
        self.text_embedding = nn.Embedding(text_vocab_size, embedding_dim)
        # self.text_embedding = 
        
        # Embedding layer for POS field using default PyTorch embeddings
        self.pos_embedding = nn.Embedding(pos_vocab_size, embedding_dim)
        
        # Two-layer LSTM
        self.lstm_pos = nn.LSTM(input_size=embedding_dim, 
                            hidden_size=hidden_dim, 
                            num_layers=num_layers, 
                            dropout=dropout,bidirectional=True)
        
        self.lstm_text = nn.LSTM(input_size=embedding_dim, 
                            hidden_size=hidden_dim, 
                            num_layers=num_layers, 
                            dropout=dropout,bidirectional=True)
        
        # Linear layer to output predictions
        self.fc = nn.Linear(2*hidden_dim, output_size)
        
        # Dropout layer
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, pos):
        # Embed text and POS tags
        # x = self.embedding(text).permute(1, 0, 2)
        text_embedded = self.text_embedding(text).permute(1, 0, 2)
        pos_embedded = self.pos_embedding(pos).permute(1, 0, 2)

        # print(text_embedded.size())
        # print(pos_embedded.size())
        
        # Pass text and POS embeddings through separate LSTM layers
        output_text, (hidden_text, cell_text) = self.lstm_text(text_embedded)
        output_pos, (hidden_pos, cell_pos) = self.lstm_pos(pos_embedded)

        # last_hidden_state = self.get_last_hidden_state(output_text)
        # print(output_text.size())
        # print(output_pos.size())

        last_hidden_state_text = self.get_last_hidden_state(output_text)
        last_hidden_state_pos = self.get_last_hidden_state(output_pos)

        # print(last_hidden_state_text.size())
        # print(last_hidden_state_pos.size())

        
        # # Apply dropout to LSTM output for text and POS
        # output_text = self.dropout(output_text)
        # output_pos = self.dropout(output_pos)

        
        
        # Pass through linear layer
        prediction_text = self.fc(last_hidden_state_text)
        prediction_pos = self.fc(last_hidden_state_pos)
        
        return prediction_text, prediction_pos
    def get_last_hidden_state(self, lstm_out):
        # Extract the last hidden state of the LSTM
        last_hidden_state_fw = lstm_out[-1, :, :self.hidden_dim]
        last_hidden_state_bw = lstm_out[0, :, self.hidden_dim:]
        last_hidden_state = torch.cat((last_hidden_state_fw, last_hidden_state_bw), dim=-1)
        return last_hidden_state




In [26]:
# Train function
def train(model, dataloader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    for pos_indices,text_indices, labels in dataloader:
        pos_indices, labels = pos_indices.to(device), labels.to(device)
        text_indices = text_indices.to(device)
        # print(pos_indices.size())
        optimizer.zero_grad()
        outputs_text,output_pos = model(pos_indices,text_indices)
        # print(outputs_text.size())
        # print(output_pos.size())
        loss = criterion(outputs_text,output_pos,labels)
        loss_text = F.cross_entropy(outputs_text.view(-1, outputs_text.size(-1)), labels.view(-1))    
        # Compute cross-entropy loss for POS prediction
        loss_pos = F.cross_entropy(output_pos.view(-1, output_pos.size(-1)), labels.view(-1))
        if (loss_text.item()<loss_pos.item()):
            predictions = torch.argmax(outputs_text, dim=1)
            correct_predictions += torch.sum(predictions == labels)
            total_predictions += len(labels)
        else:
            predictions = torch.argmax(output_pos, dim=1)
            correct_predictions += torch.sum(predictions == labels)
            total_predictions += len(labels)

        # loss = criterion(outputs, labels.squeeze())
        
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    accuracy = correct_predictions / total_predictions
    return running_loss / len(dataloader), accuracy*100

def evaluate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    with torch.no_grad():
        for pos_indices, text_indices, labels in dataloader:
            pos_indices, labels = pos_indices.to(device), labels.to(device)
            text_indices = text_indices.to(device)
            outputs_text, output_pos = model(pos_indices, text_indices)
            loss = criterion(outputs_text, output_pos, labels)
            running_loss += loss.item()
            loss_text = F.cross_entropy(outputs_text.view(-1, outputs_text.size(-1)), labels.view(-1))    
            # Compute cross-entropy loss for POS prediction
            loss_pos = F.cross_entropy(output_pos.view(-1, output_pos.size(-1)), labels.view(-1))
            if (loss_text.item()<loss_pos.item()):
                predictions = torch.argmax(outputs_text, dim=1)
                correct_predictions += torch.sum(predictions == labels)
                total_predictions += len(labels)
            else:
                predictions = torch.argmax(output_pos, dim=1)
                correct_predictions += torch.sum(predictions == labels)
                total_predictions += len(labels)
    accuracy = correct_predictions / total_predictions
    return running_loss / len(dataloader), accuracy



In [27]:
text_vocab_size = len(glove_vocab)
pos_vocab_size = len(pos_tags)
embedding_dim =  50
hidden_dim = 128
num_layers = 2
dropout = 0.5
output_size = 2

In [28]:
model = POS_GloVe_LSTM(text_vocab_size, pos_vocab_size, embedding_dim, hidden_dim, num_layers, dropout, output_size)

In [29]:
optimizer = optim.Adam(model.parameters())
criterion = CustomLoss()
batch_size = 16

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create data loaders for training and validation sets
batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,collate_fn=collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size,collate_fn=collate_fn)




# train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,collate_fn=lambda batch: collate_fn(batch, glove_vocab, pos_to_index))
# val_dataloader = DataLoader(val_dataset, batch_size=batch_size,collate_fn=lambda batch: collate_fn(batch, glove_vocab, pos_to_index))


In [30]:
# Train the model
# num_epochs = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# for epoch in range(num_epochs):
#     train_loss = train(model, train_dataloader, optimizer, criterion, device)
#     val_acc = evaluate(model, val_dataloader, criterion,device)
#     print("Epoch {}/{} - Train Loss: {:.4f} - Val Acc: {:.4f}".format(epoch+1, num_epochs, train_loss, 25))


In [31]:
num_epochs=30
for epoch in range(num_epochs):
    train_loss, train_acc = train(model, train_dataloader, optimizer, criterion, device)
    val_loss, val_acc = evaluate(model, val_dataloader, criterion,device)
    print("Epoch {}/{} - Train Loss: {:.4f} - Train Acc: {:.4f} - Val Loss: {:.4f} - Val Acc: {:.4f}".format(epoch+1, num_epochs, 
                                                                                                             train_loss, train_acc,
                                                                                                       val_loss, val_acc      ))


Epoch 1/30 - Train Loss: 1.1503 - Train Acc: 72.6581 - Val Loss: 1.0507 - Val Acc: 0.7611
Epoch 2/30 - Train Loss: 0.9392 - Train Acc: 81.0890 - Val Loss: 0.9326 - Val Acc: 0.8126
Epoch 3/30 - Train Loss: 0.8044 - Train Acc: 84.8946 - Val Loss: 0.8512 - Val Acc: 0.8454
Epoch 4/30 - Train Loss: 0.7072 - Train Acc: 86.5925 - Val Loss: 0.8092 - Val Acc: 0.8618
Epoch 5/30 - Train Loss: 0.6153 - Train Acc: 89.4614 - Val Loss: 0.8193 - Val Acc: 0.8595
Epoch 6/30 - Train Loss: 0.5725 - Train Acc: 90.6323 - Val Loss: 0.7652 - Val Acc: 0.8712
Epoch 7/30 - Train Loss: 0.5127 - Train Acc: 92.5059 - Val Loss: 0.7547 - Val Acc: 0.8712
Epoch 8/30 - Train Loss: 0.4660 - Train Acc: 93.0913 - Val Loss: 0.8503 - Val Acc: 0.8571
Epoch 9/30 - Train Loss: 0.4138 - Train Acc: 94.4965 - Val Loss: 0.8752 - Val Acc: 0.8548
Epoch 10/30 - Train Loss: 0.3763 - Train Acc: 95.6674 - Val Loss: 0.9261 - Val Acc: 0.8501
Epoch 11/30 - Train Loss: 0.3343 - Train Acc: 95.8431 - Val Loss: 0.8015 - Val Acc: 0.8806
Epoch 12

In [35]:
train_acc = evaluate(model, train_dataloader, criterion,device)

In [36]:
train_acc

(0.045699734806527045, tensor(0.9959, device='cuda:0'))

In [37]:
torch.save(model.state_dict(), "/content/gdrive/MyDrive/posglovemodeltask2.pt")

In [ ]:
# num_pos_tags = len(pos_to_index)
# num_labels = len(label_to_index)
# hidden_size = 128
# num_layers = 2
# model = POSBiLSTM(num_pos_tags, num_labels, hidden_size, num_layers,glove_vectors)

# # Define the loss function and optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters())






In [ ]:

# class POSBiLSTM(nn.Module):
#     def __init__(self, num_pos_tags, num_labels, hidden_size, num_layers, glove_embeddings):
#         super(POSBiLSTM, self).__init__()
#         self.pos_embedding = nn.Embedding(num_pos_tags, hidden_size)
#         self.glove_embeddings = glove_embeddings
#         self.lstm = nn.LSTM(input_size=hidden_size + 50 , hidden_size=hidden_size, 
#                             num_layers=num_layers, bidirectional=True)
#         self.fc = nn.Linear(hidden_size*2, num_labels)
        
#     def forward(self, pos_indices, text_indices):
#         pos_embedded = self.pos_embedding(pos_indices).permute(1, 0, 2)  # shape: (seq_len, batch_size, hidden_size)
#         text_embedded = self.glove_embeddings[text_indices].permute(1, 0, 2)  # shape: (seq_len, batch_size, glove_dim)
#         x = torch.cat((pos_embedded, text_embedded), dim=-1)  # shape: (seq_len, batch_size, hidden_size + glove_dim)
#         lstm_out, _ = self.lstm(x)  # lstm_out has shape (seq_len, batch_size, hidden_size*num_directions)
#         last_hidden_state = self.get_last_hidden_state(lstm_out)
#         out = self.fc(last_hidden_state)  # shape: (batch_size, num_labels)
#         return out
    
#     def get_last_hidden_state(self, lstm_out):
#         # Extract the last hidden state of the LSTM
#         last_hidden_state_fw = lstm_out[-1, :, :self.lstm.hidden_size]
#         last_hidden_state_bw = lstm_out[0, :, self.lstm.hidden_size:]
#         last_hidden_state = torch.cat((last_hidden_state_fw, last_hidden_state_bw), dim=-1)
#         return last_hidden_state


In [ ]:
# # Train function
# def train(model, dataloader, optimizer, criterion, device):
#     model.train()
#     running_loss = 0.0
#     for pos_indices, labels in dataloader:
#         pos_indices, labels = pos_indices.to(device), labels.to(device)
#         # print(pos_indices.size())
#         optimizer.zero_grad()
#         outputs = model(pos_indices)
#         # print(outputs.size())
#         # print(labels.size())
#         loss = criterion(outputs, labels.squeeze())
#         loss.backward()
#         optimizer.step()
#         running_loss += loss.item()
#     return running_loss / len(dataloader)

# # Evaluate function
# def evaluate(model, dataloader, device):
#     model.eval()
#     num_correct = 0
#     num_total = 0
#     with torch.no_grad():
#         for pos_indices, labels in dataloader:
#             pos_indices, labels = pos_indices.to(device), labels.to(device)
#             outputs = model(pos_indices)
#             _, predicted = torch.max(outputs.data, 1)
#             num_correct += (predicted == labels.squeeze()).sum().item()
#             num_total += pos_indices.size(0)
#     return num_correct / num_total


In [ ]:
# # Split the dataset into training and validation sets
# train_size = int(0.8 * len(dataset))
# val_size = len(dataset) - train_size
# train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# # Create data loaders for training and validation sets
# batch_size = 16
# train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,collate_fn=lambda batch: collate_fn(batch, glove_vocab, pos_to_index))
# val_dataloader = DataLoader(val_dataset, batch_size=batch_size,collate_fn=lambda batch: collate_fn(batch, glove_vocab, pos_to_index))


In [ ]:
# # Train the model
# num_epochs = 1
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)
# for epoch in range(num_epochs):
#     train_loss = train(model, train_dataloader, optimizer, criterion, device)
#     val_acc = evaluate(model, val_dataloader, device)
#     print("Epoch {}/{} - Train Loss: {:.4f} - Val Acc: {:.4f}".format(epoch+1, num_epochs, train_loss, val_acc))


In [ ]:
# glove_vocab['<unk>']

In [ ]:
# for epoch in range(num_epochs):
#     train_loss = train(model, train_dataloader, optimizer, criterion, device)
#     val_acc = evaluate(model, val_dataloader, device)
#     print("Epoch {}/{} - Train Loss: {:.4f} - Val Acc: {:.4f}".format(epoch+1, num_epochs, train_loss, val_acc))


In [ ]:
# train_acc = evaluate(model, train_dataloader, device)

In [ ]:
torch.save(model.state_dict(), "/content/gdrive/MyDrive/posglovemodeltask3.pt")


In [ ]:
# train_acc